## Load necessary modules

In [6]:
import os

def makedirs(path):
    # Intended behavior: try to create the directory,
    # pass if the directory exists already, fails otherwise.
    # Meant for Python 2.7/3.n compatibility.
    try:
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise

def get_session():
    """ Construct a modified tf session.
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)


## Configurable Params

In [9]:
class Args:
    # Paths
    local_train_dir = os.path.join(os.path.dirname((os.getcwd())), 's3_training_data')
    annotations="data/annotations.csv" ## Path to CSV file containing annotation for training
    classes="data/classmap.csv"  ## Path to CSV containing class label mapping
    val_annotations="data/val_annotations.csv"
    
    # Network
    backbone = "resnet50"
    snapshot_path = 'snapshots'

    # Hyperparameters
    batch_size= 1 
    epochs = 50 
    steps = 10000 #steps per epoch
    
    # GPU
    multi_gpu = 0 # Number of GPUs for parallel processing
    gpu = None #ID of GPU as reported by nvidia-smi
    
    # Training variables
    folder_names = ["NYR-BOS_22m12s-22m30s", "PHI-PIT_6m-8m"]
    s3_bucket_name = "bsivisiondata"
    update_csv_annotations = False
    snapshot = None #Starting point for training
    imagenet_weights = True
    
    
    # keras-retinanet specific
    evaluation = True
    dataset_type = "csv"
    snapshots = True #if true save snapshots
    weights = None
    random_transform = None
    image_min_side = 800
    image_max_side = 1333
    tensorboard_dir = None
    freeze_backbone = False
    
    
args = Args()

## Load Generators

In [8]:
# Download S3 dataset
from utils.file_manager import FileManager
fm = FileManager('bsivisiondata')
fm.download_dir('', local=args.local_train_dir)
print("S3 dataset downloaded")

Downloaded 0/150 files from DET-NSH_0h10m12s-0h10m17s/annotations/
Downloaded 0/149 files from DET-NSH_0h10m12s-0h10m17s/frames/
Downloaded 0/0 files from DET-NSH_0h10m12s-0h10m17s/
Downloaded 0/597 files from DET-NSH_0h7m45s-0h8m5s/annotations/
Downloaded 0/600 files from DET-NSH_0h7m45s-0h8m5s/frames/
Downloaded 0/0 files from DET-NSH_0h7m45s-0h8m5s/
Downloaded 0/184 files from NYR-BOS_22m12s-22m30s/annotations/
Downloaded 0/285 files from NYR-BOS_22m12s-22m30s/frames/
Downloaded 0/1 files from NYR-BOS_22m12s-22m30s/
Downloaded 0/162 files from PHI-PIT_6m-8m/annotations/
Downloaded 0/164 files from PHI-PIT_6m-8m/frames/
Downloaded 0/3 files from PHI-PIT_6m-8m/
Downloaded 0/4 files from 
S3 dataset downloaded


In [16]:
# Creates CSV file with photo annotations; filepath,x1,y1,x2,y2,class
if args.update_csv_annotations or True:
    from keras_retinanet.gen_csv_from_annotations import write_data_to_CSV
    write_data_to_CSV(args.folder_names, args.s3_bucket_name, args.local_train_dir)
print("CSV Ready")

Done.
CSV Ready


In [17]:
from keras_retinanet.bin.train import create_generators


# create the generators
train_generator, validation_generator = create_generators(args)

print("Done.")

Done.


## Load Model

In [19]:
from keras_retinanet.bin.train import create_models, check_keras_version
from keras_retinanet import models
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

# create object that stores backbone information
backbone = models.backbone(args.backbone)

# make sure keras is the minimum required version
check_keras_version()


# optionally choose specific GPU
if args.gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
set_session(get_session())

# create the model
if args.snapshot is not None:
    model            = models.load_model(args.snapshot, backbone_name=args.backbone)
    training_model   = model
    prediction_model = retinanet_bbox(model=model)
else:
    weights = args.weights
    # default to imagenet if nothing else is specified
    if weights is None and args.imagenet_weights:
        weights = backbone.download_imagenet()
        model, training_model, prediction_model = create_models(
            backbone_retinanet=backbone.retinanet,
            num_classes=train_generator.num_classes(),
            weights=weights,
            multi_gpu=args.multi_gpu,
            freeze_backbone=args.freeze_backbone
        )

print("Done.")

ImportError: No module named keras_resnet

# Load Callbacks

In [7]:
from keras_retinanet.bin.train import create_callbacks

# this lets the generator compute backbone layer shapes using the actual backbone model
if 'vgg' in args.backbone or 'densenet' in args.backbone:
    compute_anchor_targets = functools.partial(anchor_targets_bbox, shapes_callback=make_shapes_callback(model))
    train_generator.compute_anchor_targets = compute_anchor_targets
    if validation_generator is not None:
        validation_generator.compute_anchor_targets = compute_anchor_targets

# create the callbacks
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    validation_generator,
    args,
)
print("Done.")

Done.


/home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


## Start Training

In [ ]:
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=args.steps,
    epochs=args.epochs,
    verbose=1,
    callbacks=callbacks,
)
print("Done.")

Epoch 1/50
  260/10000 [..............................] - ETA: 2:31:27 - loss: 3.3184 - regression_loss: 2.4537 - classification_loss: 0.8647

## Analysis